# Graphs

A graph $G$ is comprised of two sets $V$ (the set of vertices) and $E$ (the set of edges $E \subseteq V \times V$). For an edge $e = (u, v)$ we say $e$ is directed from $u$ to $v$. $G$ is undirected if $\forall v, w \in V$

$$
(v, w) \in E \iff (w, v) \in E
$$

## Adjacency matrix

An adjacency matrix is an $n \times n$ matrix

$$
A = (a_{ij})_{0 \leq i, j \leq n-1}
$$

where 

$$
a_{ij} = \begin{cases}
1 & \text{if their is an edge from $i$ to $j$} \\
0 & \text{otherwise}
\end{cases}
$$

For example:

![](res/adjacency_matrix.png)

## Adjacency list

An array of lists, each entry in the array is adjacent to all elements in the list, for example:

![](res/adjacency_list.png)

## Adjacency Matrices vs Adjaceny Lists

|     | Adjacency matrix | Adjacency lists |
| --- | ---------------- | --------------- |
| Space | $\Theta(n^2)$ | $\Theta(n + m)$ |
| Time to check if $w$ is adjacent to $v$ | $\Theta(1)$ | $\Theta(1 + out(v))$ |
| Time to visit all $w$ adjacent to $v$ | $\Theta(n)$ | $\Theta(1 + out(v))$ |
| Time to visit all edges | $\Theta(n^2)$ | $\Theta(n + m)$ |

## General graph search strategy

The general strategy to transverse a graph is as follows

```
Alogrithm searchFromVertex(G, v)
    mark v
    put v onto schedule S
    while schedule S is not empty do
        remove a vertex v from S
        for all w adjacent to v do
            if w is not marked then
                mark w
                put w onto schedule S
```

We mark vertices that we have visited so we eventually terminate. `searchFromVertex` transverses all vertices connected to `v`, but if the graph has disconnected components, `searchFromVertex` wont visit them, thus we need an outer routing:

```
Algorithm search(G)
    initilize schedule S
    for all v in V do
        if v in not marked then
            searchFromVertex(G, v)
```

## Breadth first search

Breadth first search visits all neighbors of a vertex $v$ before visting the neighbors neigbors. To acheive this we use the general strategy where the schedule is a queue.

## Depth first search

Visit the first neighbor of $v$, then the first neigbor of the neigbor, and so on until you reach the max depth, then backtrack and visit the other neighbors. To acheive this we use the general stratergt where the schedule is a stack.

Since DFS uses a stack we can convert it easily into a recusive form:

```
Algorithm dfs(G)
    for all v in V do
        if v in not marked then
            dfsFromVertex(G, v)
            
Algorithm dfsFromVertex(G, v)
    mark v
    for all w adjacent to v do
        if w is not marked then
            dfsFromVertex(G, w)
```

## Runtime of DFS

First note that each vertex gets marked exactly once, thus `dfsFromVertex` is executed once on every vertex. We can then say the runtime is $\Theta(n)$ (n calls to `dfsFromVertex` plus the time for $n$ calls to `dfsFromVertex`.

`dfsFromVertex` has a runtime of $\Theta(1 + out(v))$ (since we loop over all adjacent vertices from $v$.

Finnaly we can compute the overall runtime:

$$
\begin{align}
T(n, m) &= \Theta(n) + \sum_{v \in V} \Theta(1 + out(v)) \\
&= \Theta\left(n + \sum_{v \in V} 1 + out(v)\right)\\
&= \Theta\left(n + n + \sum_{v \in V} out(n) \right) \\
&= \Theta(n + m)
\end{align}
$$

## Connected components

We can use DFS to find all the connected components of an (undirected) graph.

```
Algorithm connComp(G)
    for all v in V do
        if v is not marked marked then
            print "new component"
            ccFromVertex(G, v)
            
Algorithm ccFromVertex(G, v)
    mark v
    print v
    for all w adjacent to v do
        if v is not marked marked then
            ccFromVertex(G, v)
```

## Topological Sorting

A topological order for a graph $G$ is a total order $\prec$ of the vertex set $V$ such that for all edges $(v, w) \in E$ we have $v \prec w$. For example:

![](res/topological_sort.png)

Has a topological sort:

$$
8 \prec 6 \prec 7 \prec 9 \prec 5 \prec 4 \prec 2 \prec 0 \prec 1 \prec 3
$$

A directed graph has a topological order iff it has no cycles. To see why this is the case, pick a random vertice, if it is terminal make it the largest (in the topological sort) and remove it from the graph, otherwise pick a random neighbor. Since their are no cycles and the graph is finite you will eventually reach a terminal node. Repeat until the graph is empty.

We can use DFS to find this topological order (or if their is cycles). We will refer to $v$ as finished if `dfsFromVertex(G, v)` has been completed. Every vertice will be in 1 of 3 states:

- Not yet visited (white)
- Visited but not finished (grey)
- Finished (black)

If DFS reaches a grey vertex, then we know we have already visited it before, thus their must be a cycle, thus no topological sorting exists.

To prove this for a graph $G$ and a vertex $v$ consider the start of execution of `dfsFromVertex`. For all vertices $w$ we have:

1. If $w$ is white and reachable from $v$, then $w$ will be black before $v$.
2. If $w$ is grey, then $v$ is reachable from $w$.

We will define the order of $V$ as

$$
v \prec w \iff \text{$w$ becomes black before $v$}
$$

If $G$ is a DAG, then $\prec$ is an typological order.

Suppose $(v, w) \in E$. Consider `dfsFromVertex(G, v)`

- If $w$ is already black $v \prec w$
- Of $w$ is white then by part 1 above $w$ wil be black before $v$, thus $v \prec w$
- If $w$ is grey, then by part 2 above $v$ is reachable from $w$, thus their is a path from $w$ to $v$. The path and the edge $(v, w)$ form a cycle (contradiction)

The psudocode for this process:

```
Algorithm topSort(G)
    initialize array state (set all entrys to white)
    initialize linked list L
    for all v in V do
        if state[v] = white then
            sortFromVertex(G, v)
    print(L)
    
Algorithm sortFromVertex(G, v)
    state[v] <- grey
    for all w adjacent to v do
        if state[w] = white then
            sortFromVertex(G, w)
        else if state[w] = grey then
            print "G has a cycle"
            halt
    state[v] <- black
    L.insertFirst(v)
```